In [926]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [927]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime



In [1096]:
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, AGE, RANK AS COMPETITION_RANK, EVENT, DOB, COUNTRY, CATEGORY_EVENT, GENDER, COMPETITION, DATE
FROM `saa-analytics.results.saa_full` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='FOUL' AND RANK!='DNS' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT IS NOT NULL AND (CATEGORY_EVENT='Throw' OR CATEGORY_EVENT='Jump')
"""

athletes = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1097]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,"NGUI JO YI, JOEY",DNQ,None,13.0,Discus (1.00kg),None,None,Throw,Female,NSG,2023
1,THIRUMALAI NIVEDITA,DNQ,None,14.0,Discus (1.00kg),None,None,Throw,Female,NSG,2023
2,NISHKA SRIVASTAVA,DNQ,None,11.0,Discus (1.00kg),None,None,Throw,Female,NSG,2023
3,LIU MAN,DNQ,None,15.0,Discus (1.00kg),None,None,Throw,Female,NSG,2023
4,CHANG JIA EN,DNQ,None,12.0,Discus (1.00kg),None,None,Throw,Female,NSG,2023
...,...,...,...,...,...,...,...,...,...,...,...
4025,"Go, Sharon",6.66m,24,2,Women 16-99 Shot Put 4kg Open,1999-11-16 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
4026,"Yun Ning, Tan",9.44m,18,1,Women 16-99 Shot Put 4kg Open,2005-01-30 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
4027,"Tan, Ting Jun",11.90m,26,1,Men 16-99 Shot Put 7.26kg Open,1997-04-20 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023
4028,"Pongsree, Tanawin",10.28m,17,2,Men 16-99 Shot Put 7.26kg Open,2006-02-21 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023


In [1098]:
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m run')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110 Meter Hurdles.+', value='110m hurdles')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Hurdles.+', value='100m hurdles')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Hurdles.+', value='400m hurdles')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000m Race Walk.+', value='5000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000m Race Walk.+', value='3000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500m Race Walk.+', value='1500m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Shot Put.+', value='Shot put')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Discus.+', value='Discus throw')

mask = athletes['EVENT'].str.contains(r'Discus', na=True)
athletes.loc[mask, 'EVENT'] = 'Discus throw'

mask = athletes['EVENT'].str.contains(r'Shot', na=True)
athletes.loc[mask, 'EVENT'] = 'Shot put'

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'EVENT'] = 'Javelin throw'



athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw.+', value='Javelin throw')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump.+', value='High jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase.+', value='3000m steeplechase')

mask = athletes['EVENT'].str.contains(r'High', na=True)
athletes.loc[mask, 'EVENT'] = 'High jump'



In [1099]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,"NGUI JO YI, JOEY",DNQ,None,13.0,Discus throw,None,None,Throw,Female,NSG,2023
1,THIRUMALAI NIVEDITA,DNQ,None,14.0,Discus throw,None,None,Throw,Female,NSG,2023
2,NISHKA SRIVASTAVA,DNQ,None,11.0,Discus throw,None,None,Throw,Female,NSG,2023
3,LIU MAN,DNQ,None,15.0,Discus throw,None,None,Throw,Female,NSG,2023
4,CHANG JIA EN,DNQ,None,12.0,Discus throw,None,None,Throw,Female,NSG,2023
...,...,...,...,...,...,...,...,...,...,...,...
4025,"Go, Sharon",6.66m,24,2,Shot put,1999-11-16 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
4026,"Yun Ning, Tan",9.44m,18,1,Shot put,2005-01-30 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
4027,"Tan, Ting Jun",11.90m,26,1,Shot put,1997-04-20 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023
4028,"Pongsree, Tanawin",10.28m,17,2,Shot put,2006-02-21 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023


In [1100]:
athletes.to_csv('athletes.csv', sep=',', encoding='utf-8-sig', index=False)


In [1101]:
athletes.loc[(athletes['EVENT']=='Marathon') & (athletes['COMPETITION']!='SEA Games')]

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE


In [1102]:
credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND RANK='3' AND (CATEGORY_EVENT='Throw' OR CATEGORY_EVENT='Jump')
"""

benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1103]:
benchmarks

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
0,Farell Glen Felix Jerus,2.15 m,3,High jump,Jump,Male,SEA Games,Final
1,Patsapong Amsam-ang,5.20 m,3,Pole vault,Jump,Male,SEA Games,Final
2,Sapwaturrahman Sapwaturrahman,7.62,3,Long jump,Jump,Male,SEA Games,Final
3,Mark Harry Diones,15.7,3,Triple jump,Jump,Male,SEA Games,Final
4,Muhammad Ziyad Zolkefli,17.3,3,Shot put,Throw,Male,SEA Games,Final
5,Bandit Singhatongkul,50.02,3,Discus throw,Throw,Male,SEA Games,Final
6,Sadat Marzuqi Ajisan,59.76,3,Hammer throw,Throw,Male,SEA Games,Final
7,Agustinus Abadi Ndiken,66.2,3,Javelin throw,Throw,Male,SEA Games,Final
8,Michelle Sng Suat Li,1.73,3,High jump,Jump,Female,SEA Games,Final
9,Natalie Uy,4.00,3,Pole vault,Jump,Female,SEA Games,Final


In [1104]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [1105]:
benchmarks.drop(['NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [1106]:
benchmarks

,BENCHMARK,EVENT,GENDER
0,2.15 m,High jump,Male
1,5.20 m,Pole vault,Male
2,7.62,Long jump,Male
3,15.7,Triple jump,Male
4,17.3,Shot put,Male
5,50.02,Discus throw,Male
6,59.76,Hammer throw,Male
7,66.2,Javelin throw,Male
8,1.73,High jump,Female
9,4.00,Pole vault,Female


In [1107]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()    
    
    if any(s in string for s in l)==True:
        
        if 'm' in metric:
            
            metric=metric.replace('m', '')
            output=float(str(metric))
            
        elif 'GR' in metric:
            
            metric=metric.replace('GR', '')
            output=float(str(metric))


        else:
    
            output=float(str(metric))
        
    else:
        
        searchstring = ":"
        searchstring2 = "."
        substring=str(metric)
        count = substring.count(searchstring)
        count2 = substring.count(searchstring2)
            
        if count==0:
            output=float(substring)
                        
             
        elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                                                
            time=str(metric)
            h, m ,s = time.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
        elif (count==1 and count2==1):
            
            m,s = metric.split(':')
            output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
            
            if output==229.90:
                print(metric, m, s, output, 'here')

                     
        elif (count==1 and count2==2):
            
            metric = metric.replace(".", ":", 1)
            
            h,m,s = metric.split(':')            
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
        elif (count==2 and count2==0):
            
            h,m,s = metric.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                
    return output

In [1108]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,1]
    metric=benchmarks.iloc[rowIndex,0]
        
    out = convert_time(i, input_string, metric)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

In [1109]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric
0,2.15 m,High jump,Male,2.15
1,5.20 m,Pole vault,Male,5.20
2,7.62,Long jump,Male,7.62
3,15.7,Triple jump,Male,15.70
4,17.3,Shot put,Male,17.30
5,50.02,Discus throw,Male,50.02
6,59.76,Hammer throw,Male,59.76
7,66.2,Javelin throw,Male,66.20
8,1.73,High jump,Female,1.73
9,4.00,Pole vault,Female,4.00


In [1110]:
benchmarks['2pc']=benchmarks['Metric']*0.98
benchmarks['35pc']=benchmarks['Metric']*0.965
benchmarks['5pc']=benchmarks['Metric']*0.95

In [1111]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc
0,2.15 m,High jump,Male,2.15,2.1070,2.07475,2.0425
1,5.20 m,Pole vault,Male,5.20,5.0960,5.01800,4.9400
2,7.62,Long jump,Male,7.62,7.4676,7.35330,7.2390
3,15.7,Triple jump,Male,15.70,15.3860,15.15050,14.9150
4,17.3,Shot put,Male,17.30,16.9540,16.69450,16.4350
5,50.02,Discus throw,Male,50.02,49.0196,48.26930,47.5190
6,59.76,Hammer throw,Male,59.76,58.5648,57.66840,56.7720
7,66.2,Javelin throw,Male,66.20,64.8760,63.88300,62.8900
8,1.73,High jump,Female,1.73,1.6954,1.66945,1.6435
9,4.00,Pole vault,Female,4.00,3.9200,3.86000,3.8000


In [1112]:
#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[17, [1]]='10000m run'
#benchmarks.iloc[4, [1]]='1500m'
#benchmarks.iloc[4, [1]]='1500m'


In [1113]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc
0,2.15 m,High jump,Male,2.15,2.1070,2.07475,2.0425
1,5.20 m,Pole vault,Male,5.20,5.0960,5.01800,4.9400
2,7.62,Long jump,Male,7.62,7.4676,7.35330,7.2390
3,15.7,Triple jump,Male,15.70,15.3860,15.15050,14.9150
4,17.3,Shot put,Male,17.30,16.9540,16.69450,16.4350
5,50.02,Discus throw,Male,50.02,49.0196,48.26930,47.5190
6,59.76,Hammer throw,Male,59.76,58.5648,57.66840,56.7720
7,66.2,Javelin throw,Male,66.20,64.8760,63.88300,62.8900
8,1.73,High jump,Female,1.73,1.6954,1.66945,1.6435
9,4.00,Pole vault,Female,4.00,3.9200,3.86000,3.8000


In [1114]:
df = athletes.reset_index().merge(benchmarks.reset_index(), on=['EVENT','GENDER'], how='left')


In [1115]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc
0,0,"NGUI JO YI, JOEY",DNQ,None,13.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826
1,1,THIRUMALAI NIVEDITA,DNQ,None,14.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826
2,2,NISHKA SRIVASTAVA,DNQ,None,11.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826
3,3,LIU MAN,DNQ,None,15.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826
4,4,CHANG JIA EN,DNQ,None,12.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,4025,"Go, Sharon",6.66m,24,2,Shot put,1999-11-16 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,12.0,14.44,14.44,14.1512,13.9346,13.718
4026,4026,"Yun Ning, Tan",9.44m,18,1,Shot put,2005-01-30 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,12.0,14.44,14.44,14.1512,13.9346,13.718
4027,4027,"Tan, Ting Jun",11.90m,26,1,Shot put,1997-04-20 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,4.0,17.3,17.30,16.9540,16.6945,16.435
4028,4028,"Pongsree, Tanawin",10.28m,17,2,Shot put,2006-02-21 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,4.0,17.3,17.30,16.9540,16.6945,16.435


In [1116]:
df[df['EVENT']=='Shot put']

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc
5,5,"CHUA JIA YU, ZEPHENIA",DNQ,None,12.0,Shot put,None,None,Throw,Female,NSG,2023,12.0,14.44,14.44,14.1512,13.9346,13.718
6,6,KUNNAKKAT ARCHANA SURESH,DNQ,None,11.0,Shot put,None,None,Throw,Female,NSG,2023,12.0,14.44,14.44,14.1512,13.9346,13.718
7,7,HARSHITAA D/O GOPAAL,DNQ,None,13.0,Shot put,None,None,Throw,Female,NSG,2023,12.0,14.44,14.44,14.1512,13.9346,13.718
37,37,WONG CHOONG YIN,13.7,None,5.0,Shot put,None,None,Throw,Male,NSG,2023,4.0,17.3,17.30,16.9540,16.6945,16.435
38,38,NEO SHYH KAI,14.62,None,4.0,Shot put,None,None,Throw,Male,NSG,2023,4.0,17.3,17.30,16.9540,16.6945,16.435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,4025,"Go, Sharon",6.66m,24,2,Shot put,1999-11-16 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,12.0,14.44,14.44,14.1512,13.9346,13.718
4026,4026,"Yun Ning, Tan",9.44m,18,1,Shot put,2005-01-30 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,12.0,14.44,14.44,14.1512,13.9346,13.718
4027,4027,"Tan, Ting Jun",11.90m,26,1,Shot put,1997-04-20 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,4.0,17.3,17.30,16.9540,16.6945,16.435
4028,4028,"Pongsree, Tanawin",10.28m,17,2,Shot put,2006-02-21 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,4.0,17.3,17.30,16.9540,16.6945,16.435


In [1117]:
df['RESULT'] = athletes['RESULT'].replace(regex=r'–', value=np.NaN)


In [1118]:
for i in range(len(df)):
        
    rowIndex = df.index[i]

    input_string=df.iloc[rowIndex,5]
    
    
    metric=df.iloc[rowIndex,2]
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric=='NH':
        continue
    
        
    out = convert_time(i, input_string, metric)
         
    df.loc[rowIndex, 'RESULT_CONV'] = out

In [1119]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,"NGUI JO YI, JOEY",DNQ,None,13.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
1,1,THIRUMALAI NIVEDITA,DNQ,None,14.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
2,2,NISHKA SRIVASTAVA,DNQ,None,11.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
3,3,LIU MAN,DNQ,None,15.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
4,4,CHANG JIA EN,DNQ,None,12.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,4025,"Go, Sharon",6.66m,24,2,Shot put,1999-11-16 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,12.0,14.44,14.44,14.1512,13.9346,13.718,6.66
4026,4026,"Yun Ning, Tan",9.44m,18,1,Shot put,2005-01-30 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,12.0,14.44,14.44,14.1512,13.9346,13.718,9.44
4027,4027,"Tan, Ting Jun",11.90m,26,1,Shot put,1997-04-20 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,4.0,17.3,17.30,16.9540,16.6945,16.435,11.90
4028,4028,"Pongsree, Tanawin",10.28m,17,2,Shot put,2006-02-21 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,4.0,17.3,17.30,16.9540,16.6945,16.435,10.28


In [1120]:
df["AGE"].fillna(0, inplace=True)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1989164485.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["AGE"].fillna(0, inplace=True)


In [1121]:
df['AGE'] = df['AGE'].astype('float')

In [1122]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [1123]:
rslt_df = df.loc[(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon'))]

In [1124]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
68,68,"LOH DING RONG, ANSON",50.25,0.0,1.0,Discus throw,None,None,Throw,Male,NSG,2023,5.0,50.02,50.02,49.0196,48.26930,47.5190,50.25
71,71,TAN JUN JIE,48.38,0.0,2.0,Discus throw,None,None,Throw,Male,NSG,2023,5.0,50.02,50.02,49.0196,48.26930,47.5190,48.38
109,109,LEE YU FOONG (LI YUFENG),51.28,0.0,1.0,Discus throw,None,None,Throw,Male,NSG,2023,5.0,50.02,50.02,49.0196,48.26930,47.5190,51.28
257,257,Tan Jun Jie,53.4,0.0,1.0,Discus throw,None,None,Throw,Male,NSG,2024,5.0,50.02,50.02,49.0196,48.26930,47.5190,53.40
258,258,Wong Choong Yin,47.59,0.0,2.0,Discus throw,None,None,Throw,Male,NSG,2024,5.0,50.02,50.02,49.0196,48.26930,47.5190,47.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2652,2652,"SHIN, YUJIN",52.27m,22.0,2,Discus throw,2002-03-12 00:00:00.000,None,Throw,Female,84th Singapore Open Track & Field,2024,13.0,45.08,45.08,44.1784,43.50220,42.8260,52.27
2654,2654,"WICHAIDIT, Jiraya",53.65m,28.0,1,Javelin throw,1996-03-08 00:00:00.000,None,Throw,Female,84th Singapore Open Track & Field,2024,15.0,48.31,48.31,47.3438,46.61915,45.8945,53.65
2657,2657,"LEE, GAHUI",51.11m,26.0,2,Javelin throw,1998-07-28 00:00:00.000,None,Throw,Female,84th Singapore Open Track & Field,2024,15.0,48.31,48.31,47.3438,46.61915,45.8945,51.11
2934,2934,"Zhang, Guo Wei",2.25m,32.0,1,High jump,1991-06-04,None,Jump,Male,SMTFA Track & Field Championship 2023,2023,0.0,2.15 m,2.15,2.1070,2.07475,2.0425,2.25


In [1125]:
rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
rslt_df=rslt_df.loc[rslt_df['COMPETITION']!='SEA Games']

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/3172589589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/3172589589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/3172589589.py:3: SettingWithCopyWarning: 
A value is trying to be set on a co

In [1126]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5
68,68,"LOH DING RONG, ANSON",50.25,0.0,1.0,Discus throw,None,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,50.25,-1.2304,-1.98070,-2.7310
71,71,TAN JUN JIE,48.38,0.0,2.0,Discus throw,None,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,48.38,0.6396,-0.11070,-0.8610
109,109,LEE YU FOONG (LI YUFENG),51.28,0.0,1.0,Discus throw,None,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,51.28,-2.2604,-3.01070,-3.7610
257,257,Tan Jun Jie,53.4,0.0,1.0,Discus throw,None,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,53.40,-4.3804,-5.13070,-5.8810
258,258,Wong Choong Yin,47.59,0.0,2.0,Discus throw,None,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,47.59,1.4296,0.67930,-0.0710
297,297,"Loh Ding Rong, Anson",47.69,0.0,2.0,Discus throw,None,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,47.69,1.3296,0.57930,-0.1710
299,299,Lee Yu Foong (Li Yufeng),48.04,0.0,1.0,Discus throw,None,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,48.04,0.9796,0.22930,-0.5210
309,309,"Loh Ding Rong, Anson",17.57,0.0,1.0,Shot put,None,None,Throw,Male,...,4.0,17.3,17.30,16.9540,16.69450,16.4350,17.57,-0.6160,-0.87550,-1.1350
1166,1166,"Tan, Jun Jie",48.12m,16.0,1,Discus throw,2007-02-07,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,48.12,0.8996,0.14930,-0.6010
1171,1171,"Sng, Michelle",1.70m,36.0,1,High jump,1987-05-19,None,Jump,Female,...,8.0,1.73,1.73,1.6954,1.66945,1.6435,1.70,-0.0046,-0.03055,-0.0565


In [1127]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


In [1128]:
spex_list=pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/SPEX_CARDED_LIST.csv', encoding='latin-1')


In [1129]:
spex_list

,V1,V2,V3,V4,Level
0,Goh Chui Ling,"Goh, Chui Ling",NaN,NaN,E3
1,Michelle Sng,"Sng, Michelle",NaN,NaN,E3
2,Quek Jun Jie Calvin,"Quek, Jun Jie Calvin",NaN,NaN,E3
3,"SOH RUI YONG, GUILLAUME","Soh Rui Yong, Guillaume",NaN,NaN,E3
4,Aaron Justin tan wen jie,"Tan Wen Jie, Aaron Justin",NaN,NaN,E3P
5,DANIEL LEOW SOON YEE,"Yee, Daniel Leow Soon",NaN,NaN,E3P
6,Joshua Chua,"Chua, Joshua",NaN,NaN,E3P
7,NG ZHI RONG RYAN RAPHAEL,"Ng Zhi Rong, Ryan Raphael","Raphael, Ryan",NaN,E3P
8,Wenli Rachel,"Wenli, Rachel",NaN,NaN,E3P
9,Wong YaoHan Melvin,"Wong YaoHan, Melvin",NaN,NaN,E3P


In [1130]:
spex1 = spex_list['V1'].dropna().tolist()
spex2 = spex_list['V2'].dropna().tolist()
spex3 = spex_list['V3'].dropna().tolist()
spex4 = spex_list['V4'].dropna().tolist()


In [1131]:
spex_athletes=spex1+spex2+spex3+spex4

In [1132]:
spex_athletes

['Goh Chui Ling',
 'Michelle Sng',
 'Quek Jun Jie Calvin',
 'SOH RUI YONG, GUILLAUME',
 'Aaron Justin tan wen jie',
 'DANIEL LEOW SOON YEE',
 'Joshua Chua',
 'NG ZHI RONG RYAN RAPHAEL',
 'Wenli Rachel',
 'Wong YaoHan Melvin',
 'XANDER HO ANN HENG',
 'Veronica Shanti Pereira',
 'Ang Chen Xiang',
 'KAMPTON KAM',
 'MARC BRIAN LOUIS',
 'Mark Lee Ren',
 'Reuben Rainer Lee Siong En',
 'ELIZABETH-ANN TAN SHEE RU',
 'Thiruben Thana Rajan',
 'Bhavna Gopikrishna',
 'Chloe Chee En-Ya',
 'Conrad Kangli Emery',
 'Harry Irfan Curran',
 'Huang weijun',
 'Jayden tan',
 'Koh Shun Yi Audrey',
 'LAAVINIA D/O JAIGANTH',
 'Lim Yee Chern Clara',
 'LOH DING RONG ANSON',
 'ONG YING TAT',
 'SONG EN XU REAGAN',
 'Subaraghav hari',
 'Teh Ying Shan',
 'YAN TEO',
 'ZHONG CHUHAN',
 'ESTHER TAY SHEE WEI',
 'Faith Ford',
 'Garrett Chua Je-An',
 'Lucas Fun',
 'Goh, Chui Ling',
 'Sng, Michelle',
 'Quek, Jun Jie Calvin',
 'Soh Rui Yong, Guillaume',
 'Tan Wen Jie, Aaron Justin',
 'Yee, Daniel Leow Soon',
 'Chua, Joshua',

In [1133]:
spex_athletes_casefold=[s.casefold() for s in spex_athletes]

In [1134]:
spex_athletes_casefold

['goh chui ling',
 'michelle sng',
 'quek jun jie calvin',
 'soh rui yong, guillaume',
 'aaron justin tan wen jie',
 'daniel leow soon yee',
 'joshua chua',
 'ng zhi rong ryan raphael',
 'wenli rachel',
 'wong yaohan melvin',
 'xander ho ann heng',
 'veronica shanti pereira',
 'ang chen xiang',
 'kampton kam',
 'marc brian louis',
 'mark lee ren',
 'reuben rainer lee siong en',
 'elizabeth-ann tan shee ru',
 'thiruben thana rajan',
 'bhavna gopikrishna',
 'chloe chee en-ya',
 'conrad kangli emery',
 'harry irfan curran',
 'huang weijun',
 'jayden tan',
 'koh shun yi audrey',
 'laavinia d/o jaiganth',
 'lim yee chern clara',
 'loh ding rong anson',
 'ong ying tat',
 'song en xu reagan',
 'subaraghav hari',
 'teh ying shan',
 'yan teo',
 'zhong chuhan',
 'esther tay shee wei',
 'faith ford',
 'garrett chua je-an',
 'lucas fun',
 'goh, chui ling',
 'sng, michelle',
 'quek, jun jie calvin',
 'soh rui yong, guillaume',
 'tan wen jie, aaron justin',
 'yee, daniel leow soon',
 'chua, joshua',

In [1135]:
top_performers=rslt_df.sort_values(['NAME','Delta35'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [1136]:
top_performers

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5
2934,2934,"Zhang, Guo Wei",2.25m,32.0,1,High jump,1991-06-04,None,Jump,Male,...,0.0,2.15 m,2.15,2.1070,2.07475,2.0425,2.25,-0.1430,-0.17525,-0.2075
1836,1836,"Yee, Eric",48.38m,25.0,1,Discus throw,1999-03-01,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,48.38,0.6396,-0.11070,-0.8610
1177,1177,"Yee, Chun Wai, Eric",49.40m,24.0,1,Discus throw,1999-01-01,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,49.40,-0.3804,-1.13070,-1.8810
3459,3459,"Yap, Sean Yee",1.71m,28.0,1,High jump,1995-05-01,None,Jump,Female,...,8.0,1.73,1.73,1.6954,1.66945,1.6435,1.71,-0.0146,-0.04055,-0.0665
2570,2570,"YAP, SEAN YEE",1.68m,28.0,2,High jump,1995-01-05,None,Jump,Female,...,8.0,1.73,1.73,1.6954,1.66945,1.6435,1.68,0.0154,-0.01055,-0.0365
258,258,Wong Choong Yin,47.59,0.0,2.0,Discus throw,None,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,47.59,1.4296,0.67930,-0.0710
2654,2654,"WICHAIDIT, Jiraya",53.65m,28.0,1,Javelin throw,1996-03-08 00:00:00.000,None,Throw,Female,...,15.0,48.31,48.31,47.3438,46.61915,45.8945,53.65,-6.3062,-7.03085,-7.7555
2602,2602,"WEN, HUA-YU",7.43m,23.0,2,Long jump,2001-03-09 00:00:00.000,None,Jump,Male,...,2.0,7.62,7.62,7.4676,7.35330,7.2390,7.43,0.0376,-0.07670,-0.1910
2565,2565,"VONGSAVANH, XAIDAVANH",7.41m,23.0,3,Long jump,2000-10-08,None,Jump,Male,...,2.0,7.62,7.62,7.4676,7.35330,7.2390,7.41,0.0576,-0.05670,-0.1710
2595,2595,"UBAS, JANRY",7.58m,30.0,1,Long jump,1994-02-01 00:00:00.000,None,Jump,Male,...,2.0,7.62,7.62,7.4676,7.35330,7.2390,7.58,-0.1124,-0.22670,-0.3410


In [1137]:
spexed_list = top_performers.loc[~rslt_df['NAME'].str.casefold().isin(spex_athletes_casefold)]  # ~ means NOT IN. DROP spex carded athletes

In [1138]:
spexed_list

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5
2934,2934,"Zhang, Guo Wei",2.25m,32.0,1,High jump,1991-06-04,None,Jump,Male,...,0.0,2.15 m,2.15,2.1070,2.07475,2.0425,2.25,-0.1430,-0.17525,-0.2075
1836,1836,"Yee, Eric",48.38m,25.0,1,Discus throw,1999-03-01,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,48.38,0.6396,-0.11070,-0.8610
1177,1177,"Yee, Chun Wai, Eric",49.40m,24.0,1,Discus throw,1999-01-01,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,49.40,-0.3804,-1.13070,-1.8810
3459,3459,"Yap, Sean Yee",1.71m,28.0,1,High jump,1995-05-01,None,Jump,Female,...,8.0,1.73,1.73,1.6954,1.66945,1.6435,1.71,-0.0146,-0.04055,-0.0665
2570,2570,"YAP, SEAN YEE",1.68m,28.0,2,High jump,1995-01-05,None,Jump,Female,...,8.0,1.73,1.73,1.6954,1.66945,1.6435,1.68,0.0154,-0.01055,-0.0365
258,258,Wong Choong Yin,47.59,0.0,2.0,Discus throw,None,None,Throw,Male,...,5.0,50.02,50.02,49.0196,48.26930,47.5190,47.59,1.4296,0.67930,-0.0710
2654,2654,"WICHAIDIT, Jiraya",53.65m,28.0,1,Javelin throw,1996-03-08 00:00:00.000,None,Throw,Female,...,15.0,48.31,48.31,47.3438,46.61915,45.8945,53.65,-6.3062,-7.03085,-7.7555
2602,2602,"WEN, HUA-YU",7.43m,23.0,2,Long jump,2001-03-09 00:00:00.000,None,Jump,Male,...,2.0,7.62,7.62,7.4676,7.35330,7.2390,7.43,0.0376,-0.07670,-0.1910
2565,2565,"VONGSAVANH, XAIDAVANH",7.41m,23.0,3,Long jump,2000-10-08,None,Jump,Male,...,2.0,7.62,7.62,7.4676,7.35330,7.2390,7.41,0.0576,-0.05670,-0.1710
2595,2595,"UBAS, JANRY",7.58m,30.0,1,Long jump,1994-02-01 00:00:00.000,None,Jump,Male,...,2.0,7.62,7.62,7.4676,7.35330,7.2390,7.58,-0.1124,-0.22670,-0.3410


In [1139]:
spexed_list.sort_values(['EVENT', 'GENDER', 'Delta5'], ascending=[True, True, False], inplace=True)
spexed_list['overall_rank'] = 1
spexed_list['overall_rank'] = spexed_list.groupby(['EVENT', 'GENDER'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1222346855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spexed_list.sort_values(['EVENT', 'GENDER', 'Delta5'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1222346855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spexed_list['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1222346855.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [1140]:
spexed_list

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,overall_rank
2652,2652,"SHIN, YUJIN",52.27m,22.0,2,Discus throw,2002-03-12 00:00:00.000,None,Throw,Female,...,45.08,45.08,44.1784,43.50220,42.8260,52.27,-8.0916,-8.76780,-9.4440,1
2651,2651,"INSAENG, Subenrat",56.42m,30.0,1,Discus throw,1994-04-10 00:00:00.000,None,Throw,Female,...,45.08,45.08,44.1784,43.50220,42.8260,56.42,-12.2416,-12.91780,-13.5940,2
258,258,Wong Choong Yin,47.59,0.0,2.0,Discus throw,None,None,Throw,Male,...,50.02,50.02,49.0196,48.26930,47.5190,47.59,1.4296,0.67930,-0.0710,1
299,299,Lee Yu Foong (Li Yufeng),48.04,0.0,1.0,Discus throw,None,None,Throw,Male,...,50.02,50.02,49.0196,48.26930,47.5190,48.04,0.9796,0.22930,-0.5210,2
1166,1166,"Tan, Jun Jie",48.12m,16.0,1,Discus throw,2007-02-07,None,Throw,Male,...,50.02,50.02,49.0196,48.26930,47.5190,48.12,0.8996,0.14930,-0.6010,3
1836,1836,"Yee, Eric",48.38m,25.0,1,Discus throw,1999-03-01,None,Throw,Male,...,50.02,50.02,49.0196,48.26930,47.5190,48.38,0.6396,-0.11070,-0.8610,4
71,71,TAN JUN JIE,48.38,0.0,2.0,Discus throw,None,None,Throw,Male,...,50.02,50.02,49.0196,48.26930,47.5190,48.38,0.6396,-0.11070,-0.8610,5
2613,2613,"LEE, HYUNJAE",49.00m,36.0,2,Discus throw,1988-04-08 00:00:00.000,None,Throw,Male,...,50.02,50.02,49.0196,48.26930,47.5190,49.00,0.0196,-0.73070,-1.4810,6
2153,2153,"Lee, Yu Foong",49.33m,15.0,2,Discus throw,2009-03-16,None,Throw,Male,...,50.02,50.02,49.0196,48.26930,47.5190,49.33,-0.3104,-1.06070,-1.8110,7
1177,1177,"Yee, Chun Wai, Eric",49.40m,24.0,1,Discus throw,1999-01-01,None,Throw,Male,...,50.02,50.02,49.0196,48.26930,47.5190,49.40,-0.3804,-1.13070,-1.8810,8


In [1141]:
spexed_list.to_csv('ranked_list_distance.csv', sep=',', encoding='utf-8-sig', index=False)


In [1142]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,"NGUI JO YI, JOEY",DNQ,0.0,13.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
1,1,THIRUMALAI NIVEDITA,DNQ,0.0,14.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
2,2,NISHKA SRIVASTAVA,DNQ,0.0,11.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
3,3,LIU MAN,DNQ,0.0,15.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
4,4,CHANG JIA EN,DNQ,0.0,12.0,Discus throw,None,None,Throw,Female,NSG,2023,13.0,45.08,45.08,44.1784,43.5022,42.826,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,4025,"Go, Sharon",6.66m,24.0,2,Shot put,1999-11-16 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,12.0,14.44,14.44,14.1512,13.9346,13.718,6.66
4026,4026,"Yun Ning, Tan",9.44m,18.0,1,Shot put,2005-01-30 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,12.0,14.44,14.44,14.1512,13.9346,13.718,9.44
4027,4027,"Tan, Ting Jun",11.90m,26.0,1,Shot put,1997-04-20 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,4.0,17.3,17.30,16.9540,16.6945,16.435,11.90
4028,4028,"Pongsree, Tanawin",10.28m,17.0,2,Shot put,2006-02-21 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,4.0,17.3,17.30,16.9540,16.6945,16.435,10.28


In [837]:
# Find top performers not meeting bands


df2 = df.loc[(df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon'))]

df2['Delta2']=df2['2pc']-df2['RESULT_CONV']
df2['Delta35']=df2['35pc']-df2['RESULT_CONV']
df2['Delta5']=df2['5pc']-df2['RESULT_CONV']
df2=df2.loc[df2['COMPETITION']!='SEA Games']

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1211689108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Delta2']=df2['2pc']-df2['RESULT_CONV']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1211689108.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Delta35']=df2['35pc']-df2['RESULT_CONV']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1211689108.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [838]:
top_performers_2=df2.sort_values(['NAME','RESULT_CONV'],ascending=False).groupby('NAME').head(2) # Choose top performing event per NAME

top_performers_2

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5
3004,3004,{},3:38.72,0.0,2,4 x 400m relay,None,None,Relay,Male,...,22.0,3:08.82,188.82,192.5964,195.42870,198.2610,218.72,-26.1236,-23.29130,-20.4590
3003,3003,{},3:23.49,0.0,1,4 x 400m relay,None,None,Relay,Male,...,22.0,3:08.82,188.82,192.5964,195.42870,198.2610,203.49,-10.8936,-8.06130,-5.2290
5900,5900,"{9: 'Ng, Caitlin Shan Wen', 438: 'Seow, Kyra',...",54.77,0.0,1,4 x 100m relay,None,None,Relay,Female,...,9.0,44.58,44.58,45.4716,46.14030,46.8090,54.77,-9.2984,-8.62970,-7.9610
4088,4088,"{9: 'Kumar, Raam', 12: 'Muhammad, Faizal', 11:...",3:27.54,0.0,1,4 x 400m relay,None,None,Relay,Male,...,22.0,3:08.82,188.82,192.5964,195.42870,198.2610,207.54,-14.9436,-12.11130,-9.2790
5352,5352,"{995: 'Lee, Asher', 991: 'Yeo, Chee Hean Phili...",5:27.37,0.0,2,4 x 400m relay,None,None,Relay,Male,...,22.0,3:08.82,188.82,192.5964,195.42870,198.2610,327.37,-134.7736,-131.94130,-129.1090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4524,4524,"., Kaarthika",1:59.53,27.0,9,400m,1996-06-13,None,Sprint,Female,...,2.0,53.84,53.84,54.9168,55.72440,56.5320,119.53,-64.6132,-63.80560,-62.9980
4633,4633,"., Hafiz",3:18.97,23.0,16,800m,2000-09-18,None,Mid,Male,...,16.0,1:53.86,113.86,116.1372,117.84510,119.5530,198.97,-82.8328,-81.12490,-79.4170
2029,2029,"., Dharkshitha",12:46.45,14.0,5,Women 1500 Meter Race Walk U15,2010-04-06,None,Walk,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,766.45,NaN,NaN,NaN
5692,5692,"., Cheung Zheng",1:12.67,14.0,20,400m,2009-02-19,None,Sprint,Male,...,15.0,46.63,46.63,47.5626,48.26205,48.9615,72.67,-25.1074,-24.40795,-23.7085


In [839]:
top_performers_2.sort_values(['EVENT', 'GENDER', 'RESULT_CONV'], ascending=[True, True, False], inplace=True)
top_performers_2['overall_rank'] = 1
top_performers_2['overall_rank'] = top_performers_2.groupby(['EVENT', 'GENDER'])['overall_rank'].cumsum()

In [840]:
top_performers_2

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,overall_rank
3842,3842,"Loh, Cheyenne Xinyi",54:02.00,0.0,8,10000m run,None,None,Long,Female,...,35:31.03,2131.03,2173.6506,2205.61605,2237.5815,3242.00,-1068.3494,-1036.38395,-1004.4185,1
3843,3843,"Lee, Annette",47:04.28,0.0,7,10000m run,None,None,Long,Female,...,35:31.03,2131.03,2173.6506,2205.61605,2237.5815,2824.28,-650.6294,-618.66395,-586.6985,2
3841,3841,"Chan, Ashley",46:42.77,0.0,6,10000m run,None,None,Long,Female,...,35:31.03,2131.03,2173.6506,2205.61605,2237.5815,2802.77,-629.1194,-597.15395,-565.1885,3
3847,3847,"Tey, Bernice",46:33.15,0.0,5,10000m run,None,None,Long,Female,...,35:31.03,2131.03,2173.6506,2205.61605,2237.5815,2793.15,-619.4994,-587.53395,-555.5685,4
3625,3625,"Ng, Yew Cheo",46:18.36,37.0,4,10000m run,1986-11-30,None,Long,Female,...,35:31.03,2131.03,2173.6506,2205.61605,2237.5815,2778.36,-604.7094,-572.74395,-540.7785,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,3341,"DORAI, TRINITY SHARILYN",15.61,14.0,9,Women 80 Meter Hurdles (0.762m) U15,2010-10-16,None,Hurdles,Female,...,NaN,NaN,NaN,NaN,NaN,15.61,NaN,NaN,NaN,7
1550,1550,"Seu, Hoy Xuan Evelyn",14.90,14.0,2,Women 80 Meter Hurdles (0.762m) U15,2010-10-19,None,Hurdles,Female,...,NaN,NaN,NaN,NaN,NaN,14.90,NaN,NaN,NaN,8
3342,3342,"TAN, HIANG CHENG MATILDA",14.61,13.0,5,Women 80 Meter Hurdles (0.762m) U15,2011-04-04,None,Hurdles,Female,...,NaN,NaN,NaN,NaN,NaN,14.61,NaN,NaN,NaN,9
3344,3344,"ANG, ZI EN, STACIE",14.47,13.0,3,Women 80 Meter Hurdles (0.762m) U15,2011-04-24,None,Hurdles,Female,...,NaN,NaN,NaN,NaN,NaN,14.47,NaN,NaN,NaN,10
